In [ ]:
import meshio
import numpy as np
import os.path as path
import stripy as stripy
from scipy import ndimage
from netCDF4 import Dataset

## Create mesh with stripy

In [ ]:
grid = stripy.spherical_meshes.icosahedral_mesh(include_face_points=False, refinement_levels=11)

str_fmt = "{:25} {:9}"
print(str_fmt.format('Number of points', grid.npoints))
print(str_fmt.format('Number of cells', grid.simplices.shape[0]))

We also need to store each vertex neighbours... This is a pretty long process here it needs to be optimise I guess :-)

In [ ]:
ngbh = -np.ones((grid.npoints,6))
for k in range(grid.npoints):
    ids = grid.identify_vertex_neighbours(k)
    ngbh[k,0:len(ids)] = ids
    if k%100000==0:
        print('Compute ',k*100./41943042.,'% of the neighborhood')
np.savez_compressed('ngbh11', n=ngbh.astype(int))

## Reading paleoelevation grid

In [ ]:
elevfile = "data/scotese/60Ma.nc"
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

Define coordinates for interpolation on our mesh...

In [ ]:
# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

## Performing interplolation...

In [ ]:
elevations = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

In [ ]:
x = grid.points[:,0]*6378137.
y = grid.points[:,1]*6378137. 
h = grid.points[:,2]*6378137.+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

## Save model input mesh

In [ ]:
np.savez_compressed('data/scotese_cells_hr_60Ma', v=coords, c=grid.simplices, n=ngbh.astype(int), z=elevations)

In [ ]:
mesh = meshio.Mesh(coords, {'triangle': grid.simplices}, {'Z':elevations})
meshio.write("data/60_11.vtk", mesh)

In [ ]:
from time import clock
t0 = clock()
loaded = np.load("data/scotese_cells_hr_60Ma.npz")
coords = loaded['v']
ngbhs = loaded['n']
elev = loaded['z']
print('loading time ',clock()-t0)
print(elev.shape,ngbhs.shape,coords.shape)